In [3]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from matplotlib.colors import ListedColormap

#классификация
from sklearn.linear_model import LogisticRegression #логистическая регрессия
from sklearn.neural_network import MLPClassifier #нейронка
from sklearn.naive_bayes import GaussianNB #наивный байес
from sklearn.neighbors import KNeighborsClassifier # k ближайших соседей

#регрессия
from sklearn.linear_model import LinearRegression #линейная регрессия
from sklearn.svm import SVC #метод опорных векторов
from sklearn.metrics import mean_squared_error


#Оценка качества в задачах классификации
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


#Оценка качества в задачах регрессии
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

from sklearn import preprocessing

#import tensorflow as tf
#from keras.models import Sequential
#from keras.layers import Dense, Dropout

In [4]:
#Задача: вычисление значения параметра code на основании остальных параметров.
train_data = pd.read_csv("Набор_2_цены_квартир.csv",delimiter=';')#Меняем разделитель на ;


In [5]:
#выбираем рандомное число
my_code = "ВолковАнтон"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [6]:
# Определим размер валидационной выборки
val_size = round(0.2*len(train_data))
print(val_size)

408


In [7]:
# Создадим обучающую и валидационную выборки
random_state = my_seed
train, val = train_test_split(train_data, 
                              test_size=val_size, 
                              random_state=random_state)
print(len(train), len(val))

1632 408


In [8]:
print(train)

      price  totsp  livesp  kitsp  dist  metrdist  walk  brick  floor  code
815      94     65      45    7.0  17.0        10     1      0      0     2
1001     82     60      41    6.0  11.5         5     1      1      1     6
367     165     79      53    9.0   8.0        15     1      1      1     1
1285    131     78      46   10.0  17.0         5     1      0      1     2
1678    110     60      44    6.0   7.0         5     0      0      1     4
...     ...    ...     ...    ...   ...       ...   ...    ...    ...   ...
401      92     68      53    6.0  13.0        10     0      0      1     5
1957    102     58      38    6.0  15.5         8     1      0      1     1
516      80     52      34    6.0  17.0        10     1      0      0     4
1705    115     68      43    9.0   6.5         5     1      0      1     2
564      85     65      43    5.0   7.5         5     1      1      1     8

[1632 rows x 10 columns]


In [9]:
print(val)

      price  totsp  livesp  kitsp  dist  metrdist  walk  brick  floor  code
1861     82     60      45    6.0  13.5        10     0      0      1     6
464     120     74      45   10.0  13.0        15     1      0      1     5
245     225    101      65   15.0  10.5        20     1      1      1     3
772     110     65      45    7.0  13.5         7     1      0      1     1
1238    142     77      44   10.0   7.0        10     0      0      1     4
...     ...    ...     ...    ...   ...       ...   ...    ...    ...   ...
686     115     76      54    9.0  13.0         5     1      0      0     5
1008    138     79      47   10.0   7.0        10     0      0      1     4
128     149     78      48   13.0   4.5         5     1      1      1     8
147     145     78      50   10.0  12.0         3     1      0      1     5
1295     70     58      41    6.0  14.5        10     0      1      1     3

[408 rows x 10 columns]


In [10]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
num_columns = ['price','totsp','livesp','kitsp','dist','metrdist','walk','brick','floor']
ord_columns = ['code']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), num_columns)], remainder='passthrough')

ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['price', 'totsp', 'livesp', 'kitsp', 'dist',
                                  'metrdist', 'walk', 'brick', 'floor'])])

In [11]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_val = pd.DataFrame(ct.transform(val))

# даем название столбцам
column_names = num_columns + ord_columns
sc_train.columns = column_names
sc_val.columns = column_names

sc_train

,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,code
0,0.064706,0.184211,0.229730,0.10,1.000000,0.473684,1.0,0.0,0.0,2.0
1,0.047059,0.140351,0.175676,0.05,0.607143,0.210526,1.0,1.0,1.0,6.0
2,0.169118,0.307018,0.337838,0.20,0.357143,0.736842,1.0,1.0,1.0,1.0
3,0.119118,0.298246,0.243243,0.25,1.000000,0.210526,1.0,0.0,1.0,2.0
4,0.088235,0.140351,0.216216,0.05,0.285714,0.210526,0.0,0.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...
1627,0.061765,0.210526,0.337838,0.05,0.714286,0.473684,0.0,0.0,1.0,5.0
1628,0.076471,0.122807,0.135135,0.05,0.892857,0.368421,1.0,0.0,1.0,1.0
1629,0.044118,0.070175,0.081081,0.05,1.000000,0.473684,1.0,0.0,0.0,4.0
1630,0.095588,0.210526,0.202703,0.20,0.250000,0.210526,1.0,0.0,1.0,2.0


In [12]:
# В качестве входных параметров используем первые 9 числовых параметров,
# в качестве выходного - десятый числовой параметр.
x_train = sc_train[num_columns]
x_val = sc_val[num_columns]

y_train = (sc_train[ord_columns].values).flatten()
y_val = (sc_val[ord_columns].values).flatten()

In [13]:
y_train

array([2., 6., 1., ..., 4., 2., 8.])

In [15]:
#Создаем модели
models = [MLPClassifier(),
              SVC()]

In [16]:
#Обучаем модели
for model in models:
    print(model)
    model.fit(x_train, y_train)

MLPClassifier()
SVC()


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [15]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
AccuracyScore = []
PrecisionScore = []
RecallScore = []

for model in models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    AS = accuracy_score(y_val, val_pred)
    ps = precision_score(y_val, val_pred, average='weighted',zero_division = 1)
    rs = recall_score(y_val, val_pred, average='weighted',zero_division = 1)
    
    f1s.append(f1)
    AccuracyScore.append(AS)
    PrecisionScore.append(ps)
    RecallScore.append(rs)
    print(model, '\t', f1,'\t','f1')
    print(model, '\t', AS,'\t','AccuracyScore')
    print(model, '\t', ps,'\t','PrecisionScore')
    print(model, '\t', rs,'\t','RecallScore')

MLPClassifier() 	 0.3554908020784704 	 f1
MLPClassifier() 	 0.38480392156862747 	 AccuracyScore
MLPClassifier() 	 0.36653825301996834 	 PrecisionScore
MLPClassifier() 	 0.38480392156862747 	 RecallScore
SVC() 	 0.3464719654221578 	 f1
SVC() 	 0.38480392156862747 	 AccuracyScore
SVC() 	 0.3754677441353255 	 PrecisionScore
SVC() 	 0.38480392156862747 	 RecallScore


In [17]:
#выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = models[i_max]
print(best_c_model,'\t','f1')

MLPClassifier() 	 f1


In [18]:
i_max = AccuracyScore.index(max(AccuracyScore))
best_c_model = models[i_max]
print(best_c_model,'\t','AccuracyScore')

MLPClassifier() 	 AccuracyScore


In [19]:
i_max = PrecisionScore.index(max(PrecisionScore))
best_c_model = models[i_max]
print(best_c_model,'\t','PrecisionScore')

SVC() 	 PrecisionScore


In [20]:
i_max = RecallScore.index(max(RecallScore))
best_c_model = models[i_max]
print(best_c_model,'\t','RecallScore')

MLPClassifier() 	 RecallScore
